In [1]:
sentences1 = ["私は公園で遊びます。", "彼女は本を読みます。", "彼はテニスをします。", "私たちは映画を見に行きます。", "私は日本語を勉強しています。"]

sentences2 = ["私は公園で遊びます。", "私は公園で楽しみます。", "私は公園で転びます。", "私は公園で騒ぎます", "私は公園で歩きます。"]


In [2]:
sentences1 = ["私は公園で遊びます。 彼女は本を読みます。", "彼はテニスをします。 私たちは映画を見に行きます。", "私は公園で騒ぎます。 私は日本語を勉強しています。"]

sentences2 = ["私は公園で遊びます。 私は公園で楽しみます。", "私は公園で転びます。 私は公園で騒ぎます", "私は公園で転びます。 私は公園で歩きます。"]

- pre-trained BERT
- word2vec
- doc2vec
- sentence BERT

In [3]:
# pre-trained BERT

# import numpy as np
# import torch
# from transformers import BertJapaneseTokenizer, BertModel

# BERT_MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
# tokenizer = BertJapaneseTokenizer.from_pretrained(BERT_MODEL_NAME)
# model_bert = BertModel.from_pretrained(BERT_MODEL_NAME)

# def sentence_to_vector_bert(model_bert, tokenizer, sentence):
#     tokens = tokenizer(sentence, add_special_tokens=True)["input_ids"]
#     input = torch.tensor(tokens).unsqueeze(0)
#     with torch.no_grad():
#         outputs = model_bert(input, output_hidden_states=True)
#         last_hidden_state = outputs[0][:, 0, :]
#         averaged_hidden_state = last_hidden_state.mean(
#             dim=0).unsqueeze(0).reshape(-1)
#     return averaged_hidden_state


In [4]:
# sentence_to_vector_bert(model_bert, tokenizer, sentences1[0])

In [5]:
# word2vec

import re
import csv
import numpy as np
import MeCab
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

model_dir = '/Users/iomacbookair2/Documents/lab/DEIM2023/entity_vector/entity_vector.model.bin'
model_word2vec = KeyedVectors.load_word2vec_format(model_dir, binary=True)

# http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/


In [6]:

def calc_similarity_word2vec(sentence1, sentence2, model):
  mecab = MeCab.Tagger()

  pre_sentence1 = sentence1.split(" ")
  sentence1_words = []
  for s1 in pre_sentence1:
    sentence1_words += [line.split("\t")[0]
                        for line in mecab.parse(s1).split("\n")[:-2]]
  print(f"sentence1_words: {sentence1_words}")

  pre_sentence2 = sentence2.split(" ")
  sentence2_words = []
  for s2 in pre_sentence2:
    sentence2_words += [line.split("\t")[0]
                        for line in mecab.parse(s2).split("\n")[:-2]]
  print(f"sentence2_words: {sentence2_words}")

  if not sentence1_words or not sentence2_words:
    similarity = 0
  elif (len(sentence1_words) == 1 and len(sentence2_words) != 1) or (len(sentence2_words) == 1 and len(sentence1_words) != 1):
    similarity = 0
  elif all(re.match(r'.*[a-zA-Z].*', word) for word in sentence1_words) or all(re.match(r'.*[a-zA-Z].*', word) for word in sentence2_words):
    similarity = 0
  else:
    # Compute word embeddings for each sentence
    sentence1_embedding = np.mean([model[word]
                                  for word in sentence1_words if word in model], axis=0)
    print(f"sentence1_embedding: {sentence1_embedding}")
    sentence2_embedding = np.mean([model[word]
                                  for word in sentence2_words if word in model], axis=0)
    print(f"sentence2_embedding: {sentence2_embedding}")
    if sentence1_embedding is None or sentence2_embedding is None:
      similarity = 0
    else:
      similarity = cosine_similarity([sentence1_embedding], [sentence2_embedding])[0][0]

  print(f"similarity: {similarity}")

  return similarity


def calc_average_similarity_word2vec(sentences):
    similarities = []
    for i in range(len(sentences)):
        for j in range(i + 1, len(sentences)):
            similarity = calc_similarity_word2vec(
                sentences[i], sentences[j], model_word2vec)
            similarities.append(similarity)

    q = [0, 0.25, 0.5, 0.75, 1]
    outputs = {}
    for i in range(len(q)):
        outputs[f"quantile_{i}"] = np.quantile(similarities, q[i])
    outputs["average_similarity"] = np.mean(similarities)
    outputs["size"] = len(list(np.sort(np.array(similarities))))
    outputs["width_q"] = outputs["quantile_3"] - outputs["quantile_1"]

    return outputs


In [7]:
sentences1 = ["embeddng", "embeddng"]


In [8]:
calc_average_similarity_word2vec(sentences1)

sentence1_words: ['embeddng']
sentence2_words: ['embeddng']
similarity: 0


{'quantile_0': 0,
 'quantile_1': 0.0,
 'quantile_2': 0.0,
 'quantile_3': 0.0,
 'quantile_4': 0,
 'average_similarity': 0.0,
 'size': 1,
 'width_q': 0.0}

In [9]:
calc_average_similarity_word2vec(sentences2)

sentence1_words: ['私', 'は', '公園', 'で', '遊び', 'ます', '。', '私', 'は', '公園', 'で', '楽しみ', 'ます', '。']
sentence2_words: ['私', 'は', '公園', 'で', '転び', 'ます', '。', '私', 'は', '公園', 'で', '騒ぎ', 'ます']
sentence1_embedding: [ 0.06909613 -0.33147642 -0.7548873  -0.66168505  0.584137    0.40847334
 -0.3665347   0.45690754 -0.4007096  -1.1366308  -0.22266626  0.03541454
 -2.1320274   0.58637017  0.6646518   0.7895716   0.02822517  0.34234172
  1.0195876   0.02845592 -0.12776124  1.1733253   1.049341    0.2192236
  1.2635185   1.8199285   0.88672036 -0.21529952 -1.2820864   1.0999153
  0.34047598  0.85853153 -0.49414316  1.6573255   0.25328565 -0.18880764
 -2.0757606   0.6449185   2.0772684  -0.27149802  1.2181581  -1.1001714
 -0.74073136 -0.8660206   0.69807214  2.1079116   0.52865094 -0.614222
 -0.5928847  -0.13228726  0.50828844 -1.2743969   0.5019585   0.61725366
  0.81402934 -1.1782452  -0.02315041  0.9483862   0.17407167  0.9561802
 -0.27354175  0.2029345   1.3366311  -1.0055907  -0.70237285 -1.224403


{'quantile_0': 0.9716927,
 'quantile_1': 0.9763145446777344,
 'quantile_2': 0.9809364080429077,
 'quantile_3': 0.9834160506725311,
 'quantile_4': 0.9858957,
 'average_similarity': 0.9795082,
 'size': 3,
 'width_q': 0.007101505994796753}